In [ ]:
pip install qiskit

In [ ]:
import itertools
from pprint import pprint
from qiskit import QuantumCircuit, Aer, execute
backend = Aer.get_backend('qasm_simulator')  

maxgc = 4
gs = ['x','ccx']
qbnos = 4
qb = list(range(0,qbnos))

In [ ]:
qb

[0, 1, 2, 3]

In [ ]:
def toStr(n,base):
    '''
    Convert a decimal number to base-n
    '''
    convertString = "0123456789ABCDEF"
    if n < base:
        return convertString[n]
    else:
        return toStr(n//base,base) + convertString[n%base]

In [ ]:
toStr(15,16)

'F'

In [ ]:
def X_choice(opcode):
    c = list(itertools.combinations(qb, 1))
    #print(c)
    allc = []
    for i in c:
        allc.append(opcode+str(i[0]))
    return allc

In [ ]:
def CCX_choice(opcode):
    c = list(itertools.combinations(qb, 3))
    allc = []
    #print(c)
    for i in c:
        allc.append(opcode+str(i[0])+str(i[1])+str(i[2]))
        allc.append(opcode+str(i[1])+str(i[2])+str(i[0]))
        allc.append(opcode+str(i[2])+str(i[0])+str(i[1]))
    return allc

In [ ]:
setX = X_choice('0')
setCCX = CCX_choice('1')

In [ ]:
setX

['00', '01', '02', '03']

In [ ]:
setCCX

['1012',
 '1120',
 '1201',
 '1013',
 '1130',
 '1301',
 '1023',
 '1230',
 '1302',
 '1123',
 '1231',
 '1312']

In [ ]:
def progs(gc):
    p = []
    for i in range(0,len(gs)**gc):
        gseq = toStr(i,len(gs)).zfill(gc)
        #print(gseq)
        cg = ['']          
        for j in range(0, len(gseq)):
            if gseq[j] == '0':
                g = list(itertools.product(cg, setX))
            if gseq[j] == '1':
                g = list(itertools.product(cg, setCCX))
            cg = []
            for k in g:
                cg.append(''.join(map(str, k)))
        #print(gseq,': ',cg)
        for j in cg:
            p.append(j)
    return p

In [ ]:
def state_construction(index,qc):
    for i in range(4):
        if(index[i]) == '1':
            qc.x(i)

In [ ]:
possible_states = list(itertools.product('01', repeat=4))
possible_states

[('0', '0', '0', '0'),
 ('0', '0', '0', '1'),
 ('0', '0', '1', '0'),
 ('0', '0', '1', '1'),
 ('0', '1', '0', '0'),
 ('0', '1', '0', '1'),
 ('0', '1', '1', '0'),
 ('0', '1', '1', '1'),
 ('1', '0', '0', '0'),
 ('1', '0', '0', '1'),
 ('1', '0', '1', '0'),
 ('1', '0', '1', '1'),
 ('1', '1', '0', '0'),
 ('1', '1', '0', '1'),
 ('1', '1', '1', '0'),
 ('1', '1', '1', '1')]

In [ ]:
def conv2QP(desc):
    qc_list = []
    for j in possible_states:
        qcirc = QuantumCircuit(qbnos,qbnos)
        state_construction(j,qcirc)
        i = 0
        while (i < len(desc)):
            if desc[i]=='0':
                # print('X',desc[i+1])
                qcirc.x(int(desc[i+1]))
                i+= 2
            elif desc[i]=='1':
                # print('CCX',desc[i+1],desc[i+2],desc[i+3])
                qcirc.ccx(int(desc[i+1]),int(desc[i+2]),int(desc[i+3]))
                i+= 4
        qcirc.barrier()
        for i in range(0,qbnos):
            qcirc.measure(i,i)
        qc_list.append(qcirc)
    return qc_list

In [ ]:
def runQprog(desc):
    mem_result= []
    qcirc = conv2QP(desc)
    # print(desc)
    # print(qcirc.draw())
    for i in qcirc:
        job = execute(i, backend, shots=1, memory=True)
        result = job.result()
        memory = result.get_memory()
        mem_result.append(memory[0][::-1])
    return mem_result

In [ ]:
runQprog('00')

['1000',
 '1001',
 '1010',
 '1011',
 '1100',
 '1101',
 '1110',
 '1111',
 '0000',
 '0001',
 '0010',
 '0011',
 '0100',
 '0101',
 '0110',
 '0111']

In [ ]:
ng = {}
for gc in range(1,maxgc+1):
    p = progs(gc)
    for i in p:
        op = runQprog(i)
        if op in ng:
            ng[op].append(i)
        else:
            ng[op] = [i]
    print("nGC Level",gc)
    for key, value in sorted(ng.items()):
        value.sort(key=len)
        print(key, len(value),value[0],len(value[0]))

In [ ]:
gc = 1
ng = {}
p = progs(gc)
for i in p:
    op = runQprog(i)
    ng[i] = op
print("nGC Level",gc)
# for key, value in sorted(ng.items()):
#     value.sort(key=len)
#     print(key, len(value),value[0],len(value[0]))

nGC Level 1


In [ ]:
one_gate = progs(1)

In [ ]:
l = []
for i in (one_gate):
    l.append(ng[i])

In [ ]:
l

[['1000',
  '1001',
  '1010',
  '1011',
  '1100',
  '1101',
  '1110',
  '1111',
  '0000',
  '0001',
  '0010',
  '0011',
  '0100',
  '0101',
  '0110',
  '0111'],
 ['0100',
  '0101',
  '0110',
  '0111',
  '0000',
  '0001',
  '0010',
  '0011',
  '1100',
  '1101',
  '1110',
  '1111',
  '1000',
  '1001',
  '1010',
  '1011'],
 ['0010',
  '0011',
  '0000',
  '0001',
  '0110',
  '0111',
  '0100',
  '0101',
  '1010',
  '1011',
  '1000',
  '1001',
  '1110',
  '1111',
  '1100',
  '1101'],
 ['0001',
  '0000',
  '0011',
  '0010',
  '0101',
  '0100',
  '0111',
  '0110',
  '1001',
  '1000',
  '1011',
  '1010',
  '1101',
  '1100',
  '1111',
  '1110'],
 ['0000',
  '0001',
  '0010',
  '0011',
  '0100',
  '0101',
  '0110',
  '0111',
  '1000',
  '1001',
  '1010',
  '1011',
  '1110',
  '1111',
  '1100',
  '1101'],
 ['0000',
  '0001',
  '0010',
  '0011',
  '0100',
  '0101',
  '1110',
  '1111',
  '1000',
  '1001',
  '1010',
  '1011',
  '1100',
  '1101',
  '0110',
  '0111'],
 ['0000',
  '0001',
  '0010',
  '0

In [ ]:
import numpy as np

In [ ]:
possible_states = ['0000',
  '0001',
  '0010',
  '0011',
  '0100',
  '0101',
  '0110',
  '0111',
  '1000',
  '1001',
  '1010',
  '1011',
  '1100',
  '1101',
  '1110',
  '1111']

In [ ]:
# short circuits at shortest nested list if table is jagged:
list(map(list, zip(*l)))

# discards no data if jagged and fills short nested lists with None
transpose_list = list(map(list, itertools.zip_longest(*l, fillvalue=None)))

In [ ]:
all_first_layer_list={}
for i in range(qbnos**2):
    all_first_layer_list[possible_states[i]] = list(np.unique(transpose_list[i]))

In [ ]:
all_first_layer_list

{'0000': ['0000', '0001', '0010', '0100', '1000'],
 '0001': ['0000', '0001', '0011', '0101', '1001'],
 '0010': ['0000', '0010', '0011', '0110', '1010'],
 '0011': ['0001', '0010', '0011', '0111', '1011'],
 '0100': ['0000', '0100', '0101', '0110', '1100'],
 '0101': ['0001', '0100', '0101', '0111', '1101'],
 '0110': ['0010', '0100', '0110', '0111', '1110'],
 '0111': ['0011', '0101', '0110', '0111', '1111'],
 '1000': ['0000', '1000', '1001', '1010', '1100'],
 '1001': ['0001', '1000', '1001', '1011', '1101'],
 '1010': ['0010', '1000', '1010', '1011', '1110'],
 '1011': ['0011', '1001', '1010', '1011', '1111'],
 '1100': ['0100', '1000', '1100', '1101', '1110'],
 '1101': ['0101', '1001', '1100', '1101', '1111'],
 '1110': ['0110', '1010', '1100', '1110', '1111'],
 '1111': ['0111', '1011', '1101', '1110']}

In [ ]:
all_first_layer_list['0000']

['0000', '0001', '0010', '0100', '1000']

In [ ]:
def create_adjacency_layer(state):
    layer = 1
    first_state_adjacency = {}
    for i in possible_states:
        first_state_adjacency[i] = 0
    for i in all_first_layer_list[state]:
        if((first_state_adjacency[i] == 0)):
            first_state_adjacency[i] = 1
    for j in possible_states:
        if((first_state_adjacency[j] == 1) & (j != state)):
            for k in all_first_layer_list[j]:
                if(first_state_adjacency[k] == 0):
                    first_state_adjacency[k] = 2
    for j in possible_states:
        if((first_state_adjacency[j] == 2) & (j != state)):
            for k in all_first_layer_list[j]:
                if(first_state_adjacency[k] == 0):
                    first_state_adjacency[k] = 3
    for j in possible_states:
        if((first_state_adjacency[j] == 3) & (j != state)):
            for k in all_first_layer_list[j]:
                if(first_state_adjacency[k] == 0):
                    first_state_adjacency[k] = 4
    return first_state_adjacency

In [ ]:
create_adjacency_layer('0000')

{'0000': 1,
 '0001': 1,
 '0010': 1,
 '0011': 2,
 '0100': 1,
 '0101': 2,
 '0110': 2,
 '0111': 3,
 '1000': 1,
 '1001': 2,
 '1010': 2,
 '1011': 3,
 '1100': 2,
 '1101': 3,
 '1110': 3,
 '1111': 4}

In [ ]:
adjancency_matrix = []
for i in possible_states:
    adjancency_matrix.append(create_adjacency_layer(i))

In [ ]:
adjancency_matrix[0]

{'0000': 1,
 '0001': 1,
 '0010': 1,
 '0011': 2,
 '0100': 1,
 '0101': 2,
 '0110': 2,
 '0111': 3,
 '1000': 1,
 '1001': 2,
 '1010': 2,
 '1011': 3,
 '1100': 2,
 '1101': 3,
 '1110': 3,
 '1111': 4}